# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [4]:
# Data preparation

# Imports the necessary libraries
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

# Reads the dataset into a dataframe
file_path = 'tic-tac-toe.csv'  # Update this with the actual file path
data = pd.read_csv(file_path)

# Inspects the dataset
print("First few rows of the dataset:")
print(data.head())

print("\nDataset information:")
print(data.info())

print("\nUnique values in each column:")
for column in data.columns:
    print(f"{column}: {data[column].unique()}")

First few rows of the dataset:
  TL TM TR ML MM MR BL BM BR  class
0  x  x  x  x  o  o  x  o  o   True
1  x  x  x  x  o  o  o  x  o   True
2  x  x  x  x  o  o  o  o  x   True
3  x  x  x  x  o  o  o  b  b   True
4  x  x  x  x  o  o  b  o  b   True

Dataset information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 958 entries, 0 to 957
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   TL      958 non-null    object
 1   TM      958 non-null    object
 2   TR      958 non-null    object
 3   ML      958 non-null    object
 4   MM      958 non-null    object
 5   MR      958 non-null    object
 6   BL      958 non-null    object
 7   BM      958 non-null    object
 8   BR      958 non-null    object
 9   class   958 non-null    bool  
dtypes: bool(1), object(9)
memory usage: 68.4+ KB
None

Unique values in each column:
TL: ['x' 'o' 'b']
TM: ['x' 'o' 'b']
TR: ['x' 'o' 'b']
ML: ['x' 'o' 'b']
MM: ['o' 'b' 'x']
MR: ['o' 'b' 'x']


In [27]:
# Converts categorical values to numeric in all columns
# Uses LabelEncoder to encode categorical data into numeric values
label_encoders = {}
for column in data.columns:
    label_encoders[column] = LabelEncoder()
    data[column] = label_encoders[column].fit_transform(data[column])

print("\nDataset after converting categorical values to numeric:")
print(data.head())

# Separates the inputs (X) and output (y)
# Assuming the output column is the last column:
X = data.iloc[:, :-1]  # All columns except the last one
y = data.iloc[:, -1]   # The last column

print("\nInputs (X):")
print(X.head())
print("\nOutput (y):")
print(y.head())

# Normalizes the input data
scaler = MinMaxScaler()
X_normalized = scaler.fit_transform(X)

# Converts normalized data back to a dataframe
X_normalized = pd.DataFrame(X_normalized, columns=X.columns)

print("\nNormalized inputs (X):")
print(X_normalized.head())


Dataset after converting categorical values to numeric:
   TL  TM  TR  ML  MM  MR  BL  BM  BR  class
0   2   2   2   2   1   1   2   1   1      1
1   2   2   2   2   1   1   1   2   1      1
2   2   2   2   2   1   1   1   1   2      1
3   2   2   2   2   1   1   1   0   0      1
4   2   2   2   2   1   1   0   1   0      1

Inputs (X):
   TL  TM  TR  ML  MM  MR  BL  BM  BR
0   2   2   2   2   1   1   2   1   1
1   2   2   2   2   1   1   1   2   1
2   2   2   2   2   1   1   1   1   2
3   2   2   2   2   1   1   1   0   0
4   2   2   2   2   1   1   0   1   0

Output (y):
0    1
1    1
2    1
3    1
4    1
Name: class, dtype: int64

Normalized inputs (X):
    TL   TM   TR   ML   MM   MR   BL   BM   BR
0  1.0  1.0  1.0  1.0  0.5  0.5  1.0  0.5  0.5
1  1.0  1.0  1.0  1.0  0.5  0.5  0.5  1.0  0.5
2  1.0  1.0  1.0  1.0  0.5  0.5  0.5  0.5  1.0
3  1.0  1.0  1.0  1.0  0.5  0.5  0.5  0.0  0.0
4  1.0  1.0  1.0  1.0  0.5  0.5  0.0  0.5  0.0


## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [15]:
# Imports the necessary libraries
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

In [17]:
# Splits the training and test data, creates a sequential model,
# Adds several layers to the model.
# Uses Softmax for the output layer.
# Compiles the model using adam as optimizer and sparse_categorical_crossentropy as the loss
# function. Fits the training data, evaluates the nnm with the test data and saves the model.

# Separates inputs (X) and outputs (y)
X = data.iloc[:, :-1]  # All columns except the last one
y = data.iloc[:, -1]   # The last column (target)

# Normalizes the input data
scaler = MinMaxScaler()
X_normalized = scaler.fit_transform(X)

# Splits the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

In [29]:
# Builds the neural network model
model = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),  # Input layer
    Dense(16, activation='relu'),  # Hidden layer
    Dense(len(set(y)), activation='softmax')  # Output layer
])

# Compiles the model
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Trains the model
print("\nTraining the model...")
model.fit(X_train, y_train, epochs=50, batch_size=8, validation_split=0.2, verbose=1)

# Evaluates the model
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"\nTest Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")


# Saves the model in HDF5 format
model.save('tic-tac-toe.keras')

print("Model saved as tic-tac-toe.keras")


Training the model...
Epoch 1/50


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4401 - loss: 0.7346 - val_accuracy: 0.6818 - val_loss: 0.6286
Epoch 2/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 660us/step - accuracy: 0.6449 - loss: 0.6378 - val_accuracy: 0.6818 - val_loss: 0.6201
Epoch 3/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 699us/step - accuracy: 0.6776 - loss: 0.6212 - val_accuracy: 0.6948 - val_loss: 0.6148
Epoch 4/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 687us/step - accuracy: 0.6981 - loss: 0.5874 - val_accuracy: 0.7143 - val_loss: 0.6154
Epoch 5/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 646us/step - accuracy: 0.6980 - loss: 0.5924 - val_accuracy: 0.6948 - val_loss: 0.6123
Epoch 6/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 663us/step - accuracy: 0.7106 - loss: 0.5919 - val_accuracy: 0.7013 - val_loss: 0.6071
Epoch 7/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 666us/step - accuracy: 0.7185 - loss: 0.5733 - val_accuracy: 0.6818 - val_loss: 0.6097
Epoch 8/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 673us/step - accuracy: 0.7167 - loss: 0.5689 - val_accuracy: 0.6818 - val_lo

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [31]:
# Loads the saved model and uses it to make predictions on a few random rows in the test dataset. 
# Check if the predictions are correct.

# Imports the necessary libraries
from tensorflow.keras.models import load_model
import numpy as np

# Loads the saved model
model = load_model('tic-tac-toe.keras')

# Makes predictions on random rows
# Randomly select 5 rows from the test dataset
random_indices = np.random.choice(X_test.shape[0], 5, replace=False)  
X_sample = X_test[random_indices]  
y_true = y_test.iloc[random_indices]  

# Predicts the probabilities for each class
predictions = model.predict(X_sample)

# Gets the predicted class (highest probability)
y_pred = np.argmax(predictions, axis=1)

# Compares the predictions with true labels
print("\nPredictions vs True Labels:")
for i in range(len(y_true)):
    print(f"Sample {i + 1}: Predicted = {y_pred[i]}, True = {y_true.iloc[i]}")

# Evaluates the model on the entire test set 
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"\nTest Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step

Predictions vs True Labels:
Sample 1: Predicted = 0, True = 1
Sample 2: Predicted = 0, True = 0
Sample 3: Predicted = 0, True = 1
Sample 4: Predicted = 1, True = 1
Sample 5: Predicted = 1, True = 1

Test Loss: 0.4595
Test Accuracy: 0.8021


## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [35]:
# Imports the necessary libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Input

# Builds the neural network with more layers
from tensorflow.keras import Input

# Updated model definition with Input layer
model = Sequential([
    Input(shape=(X_train.shape[1],)),  # Define the input shape explicitly
    Dense(64, activation='relu'),  # First hidden layer
    Dense(32, activation='relu'),  # Second hidden layer
    Dense(16, activation='relu'),  # Third hidden layer
    Dense(len(set(y)), activation='softmax')  # Output layer
])
# Compiles the model with a custom learning rate
# Defines an Adam optimizer with a lower learning rate
custom_optimizer = Adam(learning_rate=0.001)  # Adjust learning rate as needed
model.compile(optimizer=custom_optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Trains the model with more epochs
print("\nTraining the model with more layers, adjusted learning rate, and more epochs...")
history = model.fit(X_train, y_train, epochs=100, batch_size=8, validation_split=0.2, verbose=1)

# Evaluates the model
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"\nTest Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")


Training the model with more layers, adjusted learning rate, and more epochs...
Epoch 1/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5525 - loss: 0.6745 - val_accuracy: 0.6818 - val_loss: 0.6094
Epoch 2/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 679us/step - accuracy: 0.6634 - loss: 0.6184 - val_accuracy: 0.6818 - val_loss: 0.5986
Epoch 3/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 667us/step - accuracy: 0.6483 - loss: 0.6120 - val_accuracy: 0.7143 - val_loss: 0.5964
Epoch 4/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 673us/step - accuracy: 0.7097 - loss: 0.5867 - val_accuracy: 0.6948 - val_loss: 0.6095
Epoch 5/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 868us/step - accuracy: 0.7663 - loss: 0.5347 - val_accuracy: 0.7597 - val_loss: 0.5478
Epoch 6/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 727us/step - accuracy: 0.7691 - loss: 0.5074 - val_accuracy: 0.7662 - val_loss: 0.5204
Epoch 7/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 741us/step - accuracy: 0.7575 - loss: 0.4975 - val_accuracy: 0.7597 - val_loss: 0.5107
Epoch 8/100
77/77 

**Which approach(es) did you find helpful to improve your model performance?**

- **Adding more layers** helped the model to learn more complex patterns found within the data.
- **Adjusting the learning** rate seemed to help. The model converged more steadily, reducing the probability of overshooting the optimal solution.
- **Increasing the number of epochs** was somewhat helpful. Accuracy plateaued after approximately 50 epochs. At that point the validation loss started to fluctuate, signaling possible overfitting.